First Jupyter Notebook For Debugging Purposes!

In [38]:
using Plots

# Constants: (SI Units)
const M_S = 1.989e30 # The Mass of The Sun (Kg)

1.989e30

In [39]:
const M_E = 5.972e24 # The Mass of The Earth (Kg)

5.972e24

In [40]:
const M_M = 7.34767309e22 # The Mass of The Moon (Kg)

7.34767309e22

In [41]:
const G = 6.6743e-11 # Gravitational Constant (m^3kg^-1s^-2)

6.6743e-11

In [42]:
const AU = 149597870700;

In [43]:
const SEMD = AU # Sun-Earth Mean Distance = 149597870.700 Km

149597870700

In [44]:
const EMMD = AU / 391.1055443137255 # Earth-Moon Mean Distance = 382,500 Km

3.825e8

In [45]:
const M_i = 5.15 # The Moon moves in an approximately elliptic orbit inclined 
# at about five degrees to the plane of the ecliptic. (In Degrees)

5.15

In [46]:
## Linear Velocity of The Earth In Orbit Around The Sun:
# This equation can be proved with the help of polar
# coordinates and two assumptions:
# 1- r is constant in orbital motion (r = AU)
# 2- Theta_Dot is constant in orbital motuon (Theta_Dot = cte.)
const V_E0 = (G * M_S / SEMD)^0.5 # Earth Initial Velocity

29789.111319772674

In [47]:
const V_M0 = (G * M_E / EMMD)^0.5 + V_E0 # Moon Initial Velocity

30809.92631648674

In [48]:
# State-Space Matrix Calculation Function:
function f(x_var)
    D_A1_A2 = (((x_var[1] - x_var[4])^2 + (x_var[2] - x_var[5])^2 + (x_var[3] - x_var[6])^2)^(3 / 2))
    D_B1 = ((x_var[1]^2 + x_var[2]^2 + x_var[3]^2)^(3 / 2)) # Inverse of The Distance of The Moon
    D_B2 = ((x_var[4]^2 + x_var[5]^2 + x_var[6]^2)^(3 / 2)) # Inverse of The Distance of The Earth

    A1 = (G * M_E) / D_A1_A2
    A2 = (G * M_M) / D_A1_A2
    B1 = (G * M_S) / D_B1
    B2 = (G * M_S) / D_B2

    Z1 = -B1 - A1
    Z2 = -B2 - A2

    # Stat-Space Matrix
    A = [0 0 0 0 0 0 1 0 0 0 0 0
        0 0 0 0 0 0 0 1 0 0 0 0
        0 0 0 0 0 0 0 0 1 0 0 0
        0 0 0 0 0 0 0 0 0 1 0 0
        0 0 0 0 0 0 0 0 0 0 1 0
        0 0 0 0 0 0 0 0 0 0 0 1
        Z1 0 0 A1 0 0 0 0 0 0 0 0
        0 Z1 0 0 A1 0 0 0 0 0 0 0
        0 0 Z1 0 0 A1 0 0 0 0 0 0
        A2 0 0 Z2 0 0 0 0 0 0 0 0
        0 A2 0 0 Z2 0 0 0 0 0 0 0
        0 0 A2 0 0 Z2 0 0 0 0 0 0]

    f_x = A * x_var
    return f_x
end

f (generic function with 1 method)

In [49]:
# Initialize System:
Day_Hours = 23 + (56 / 60) + (4 / 3600) # Hours of A Day

23.934444444444445

In [50]:
Year_Days = 365.2425 # Days of 

365.2425

In [51]:
Year_Seconds = Year_Days * Day_Hours * 3600

3.1470754770000003e7

In [52]:
dD = 0.1 # Simulation Step In Day

0.1

In [53]:
Stop_Year = 0.08 # The year that the simulation will stop.

0.08

In [54]:
dY = dD / 365.2425 # Simulation  Step In Years

0.00027379070069885077

In [55]:
Δt = dY * Year_Seconds # In Seconds

8616.400000000001

In [56]:
Year = 0:dY:Stop_Year # In Years

0.0:0.00027379070069885077:0.07994688460406442

In [57]:
xM0 = -SEMD - (EMMD * cosd(M_i))

-1.4997882659005237e11

In [58]:
yM0 = 0

0

In [59]:
zM0 = EMMD * sind(M_i)

3.433452831215084e7

In [60]:
VxM0 = 0

0

In [61]:
VyM0 = -V_M0

-30809.92631648674

In [62]:
VzM0 = 0

0

In [63]:
xE0 = -SEMD

-149597870700

In [64]:
yE0 = 0

0

In [65]:
zE0 = 0

0

In [66]:
VxE0 = 0

0

In [67]:
VyE0 = -V_E0

-29789.111319772674

In [68]:
VzE0 = 0

0

In [69]:
x = zeros(12, length(Year));

In [70]:
x[:, 1] = [xM0 yM0 zM0 xE0 yE0 zE0 VxM0 VyM0 VzM0 VxE0 VyE0 VzE0]'

12×1 adjoint(::Matrix{Float64}) with eltype Float64:
     -1.4997882659005237e11
      0.0
      3.433452831215084e7
     -1.495978707e11
      0.0
      0.0
      0.0
 -30809.92631648674
      0.0
      0.0
 -29789.111319772674
      0.0

In [71]:
i = 1

1

In [72]:
k1 = Δt .* f(x[:, i])

12-element Vector{Float64}:
  0.0
 -2.654706491133764e8
  0.0
  0.0
 -2.566748987756893e8
  0.0
 74.23112392427625
  0.0
 -2.1187559921394516
 50.82348784402189
  0.0
  0.0259249643746711

In [73]:
k2 = Δt .* f(x[:, i] + k1 ./ 2)

12-element Vector{Float64}:
 319802.528090567
     -2.654706491133764e8
  -9128.024565335187
 218957.75032961514
     -2.566748987756893e8
    111.68993151895806
     74.22642891737196
      0.31484984525366233
     -2.118338214900043
     50.82348845032349
      0.04052727508735539
      0.025919824404676602

In [74]:
k3 = Δt .* f(x[:, i] + k2 ./ 2)

12-element Vector{Float64}:
 319782.3010618219
     -2.654692926772731e8
  -9126.224697432368
 218957.7529416837
     -2.566747241760828e8
    111.66778750022776
     74.23272622367081
      0.31495716567464926
     -2.1188898614734857
     50.82348711361657
      0.040526052702773666
      0.025926611110917222

In [75]:
k4 = Δt .* f(x[:, i] + k3)

12-element Vector{Float64}:
 639618.8622336373
     -2.6546793531645405e8
 -18257.202602400146
 437915.4943657659
     -2.5667454958700877e8
    223.3940519761072
     74.22493792157721
      0.6297345297336068
     -2.1181881267847587
     50.823487603005326
      0.0810541540286522
      0.025917977297748477